## Imports

In [1]:
import pandas as pd
import json
from datetime import *

## Loading the data

In [2]:
# Loading sleep data from Oura json file available on https://cloud.ouraring.com/profile 
with open('/Users/user/Desktop/Msc AI RU/Internship/Code/oura_json/oura_sleep_2022-12-06T10-07-33.json') as f:
    data1 = json.load(f)

sleep_df = pd.json_normalize(data1['sleep'], max_level=2)
print('sleep_df', sleep_df.shape)

sleep_df (176, 56)


In [3]:
# Loading raw wake up data from Google Form csv
wakeup_df = pd.read_csv ('questionnaire_data/raw/Wakeup_3_v1_raw.csv')
print('wakeup_df', wakeup_df.shape)

# Loading raw bedtime data from Google Form csv
bedtime_df = pd.read_csv ('questionnaire_data/raw/Bedtime_3_v1_raw.csv')
bedtime_df.columns = ['Timestamp_bedtime' if x=='Timestamp' else x for x in bedtime_df.columns]
print('bedtime_df', bedtime_df.shape)

wakeup_df (151, 9)
bedtime_df (149, 10)


## Combining the three dataframes

In [4]:
# Finding start and end date of all dataframes
sleep_start_date = sleep_df.iloc[0]['day']
sleep_end_date = sleep_df.iloc[-1]['day']

wakeup_start_date = wakeup_df.iloc[0]['Timestamp']
wakeup_end_date = wakeup_df.iloc[-1]['Timestamp']

bedtime_start_date = bedtime_df.iloc[0]['Timestamp_bedtime']
bedtime_end_date = bedtime_df.iloc[-1]['Timestamp_bedtime']

# Only consider the date of the questionnaire data and ignores the exact time
def remove_hour(string_date):
    split_string = string_date.split(" ")
    return split_string[0]

# Change how the date of the oura data is stored
def convert_sleep_date(sleep_date):
    y, m, d = [int(x) for x in sleep_date.split('-')]
    return date(y, m, d)

# Change how the date of the questionnaire data is stored
def convert_questionnaire_date(quest_date):
    m, d, y = [int(x) for x in remove_hour(quest_date).split('/')]
    return date(y, m, d)

# Compare two dates and only return the earliest or the latest, depending on what is required
def compare_two_dates(date1, date2, earliest):
    if date1 == date2:
        earliest_date = latest_date = date1
    elif date1 > date2:
        earliest_date = date2
        latest_date = date1  
    else:
        earliest_date = date1
        latest_date = date2
    if earliest:
        return earliest_date
    else:
        return latest_date

# Compare three dates and only return the earliest or the latest, depending on what is required
def compare_three_dates(sleep_date, wakeup_date, bedtime_date, earliest):
    date1 = convert_sleep_date(sleep_date)
    date2 = convert_questionnaire_date(wakeup_date)
    date3 = convert_questionnaire_date(bedtime_date)
    
    if earliest:
        earliest_date = compare_two_dates(date1, date2, earliest)
        if earliest_date > date3:
            earliest_date = date3
        return earliest_date
    else:
        latest_date = compare_two_dates(date1, date2, earliest)
        if latest_date < date3:
            latest_date = date3
        return latest_date

# Determine the time range for which all three data sets simultaneously have data
latest_start = compare_three_dates(sleep_start_date, wakeup_start_date, bedtime_start_date, earliest = False) + timedelta(days=1)
earliest_end = compare_three_dates(sleep_end_date, wakeup_end_date, bedtime_end_date, earliest = True) + timedelta(days=-1)
print('latest date', latest_start, "\nearliest date", earliest_end)


latest date 2022-06-20 
earliest date 2022-11-18


In [5]:
# Determine the days with missing data
def find_missing_days(df, start_date, end_date, column_name, oura):
    if oura == False:
        for i in df[column_name]:
            df[column_name] = df[column_name].replace([i], convert_questionnaire_date(i))

    # Set the date values as index
    df = df.set_index(column_name)

    # Convert string format of date to a DateTime object
    df.index = pd.to_datetime(df.index)

    # Dates which are not in the sequence are returned
    return pd.date_range(start_date, end_date).difference(df.index)

In [6]:
# Create a dataframe that goes from latest_start to earliest_end with no jumps in the dates, except that days without data are empty rows.
def complete_df(df, latest_start, earliest_end, column_name, oura=False, bedtime_data=False):
    # Bedtime data of day n needs to be matched with wakeup and sleep data of day n+1
    if bedtime_data == True:
        latest_start = latest_start + timedelta(days=-1)
        earliest_end = earliest_end + timedelta(days=-1)

    missing_days = find_missing_days(df, latest_start, earliest_end, column_name, oura)
    no_jumps_df = pd.DataFrame()
    counter = 0 
    for index, row in df.iterrows():
        if oura == True:
            sleep_date = convert_sleep_date(row[column_name])
        else:
            sleep_date = row[column_name]

        # Only add the data starting the latest_start
        if counter == 0 and sleep_date < latest_start:
            continue
        
        # Prevents to add data twice for a single day
        if sleep_date > latest_start and sleep_date == latest_date:
            continue
        latest_date = sleep_date

        # If data is missing from the considered date, add an empty row
        while counter < (len(missing_days)) and sleep_date > missing_days[counter]:
            no_jumps_df = no_jumps_df.append([None])
            counter += 1

        no_jumps_df = no_jumps_df.append(pd.DataFrame(row).transpose())
        
        # End when the earliest_end is reached
        if (sleep_date == earliest_end):
            break
               
    return no_jumps_df

In [7]:
# Create three datasets that cover the same time period
complete_sleep_df = complete_df(sleep_df, latest_start, earliest_end, 'day', oura=True)
complete_wakeup_df = complete_df(wakeup_df, latest_start, earliest_end, 'Timestamp')
complete_bedtime_df = complete_df(bedtime_df, latest_start, earliest_end, 'Timestamp_bedtime', bedtime_data=True)

print('complete_sleep_df', complete_sleep_df.shape, '\ncomplete_wakeup_df', complete_wakeup_df.shape, '\ncomplete_bedtime_df', complete_bedtime_df.shape)

complete_sleep_df (152, 57) 
complete_wakeup_df (152, 10) 
complete_bedtime_df (152, 11)


In [9]:
# Concatenate the 3 data sets. Merge the different columns on to the same rows 
df = pd.concat([complete_sleep_df.reset_index(), complete_wakeup_df.reset_index(), complete_bedtime_df.reset_index()], axis=1)
df

2022-06-21
